# **Large downloads in astroquery.mast from AWS to local storage**

## Introduction

Several MAST datasets are now available from the [Registry of Open Data on AWS](https://registry.opendata.aws/), a cloud data storage service. These datasets [include data from JWST, HST, TESS, Pan-STARRS, GALEX, and Kepler/K2](https://registry.opendata.aws/collab/stsci/). In this notebook, you'll learn how to download data in bulk to your local machine's storage from two large survey missions, GALEX and Pan-STARRS (PS1).

To give some more context on the missions we'll be focusing on:
* [Galaxy Evolution Explorer (GALEX)](https://archive.stsci.edu/missions-and-data/galex) was a NASA mission led by the California Institute of Technology, whose primary goal was to investigate how star formation in galaxies evolved from the early Universe up to the present. GALEX used microchannel plate detectors to obtain direct images in the near-UV (NUV) and far-UV (FUV), and a grism to disperse light for low resolution spectroscopy. 
* [Panoramic Survey Telescope and Rapid Response System (Pan-STARRS)](https://outerspace.stsci.edu/display/PANSTARRS/) is a system for wide-field astronomical imaging developed and operated by the Institute for Astronomy at the University of Hawaii. Pan-STARRS1 (PS1) is the first part of Pan-STARRS to be completed.  The PS1 survey used a 1.8 meter telescope and its 1.4 Gigapixel camera to image the sky in five broadband filters (g, r, i, z, y).

## Learning Goals
In this notebook, you will:
* Learn how to download MAST data from our AWS cloud repositories to your local machine's storage.
* Make targeted queries to MAST using parameters such as: `right ascension`, `declination`, and more.
* Filter the resulting products by using parameters such as: `productType`, `productSubGroupDescription`, `productGroupDescription`, `mrp_only`, and more.

## Imports

We only need one import for this notebook!
* `astroquery.mast.Observations` to access the MAST API

In [ ]:
from astroquery.mast import Observations

## Using astroquery to query MAST's multi-mission database
* The `Observations` API from `astroquery.mast` can be used to query MAST's multi-mission database, an instance of the [Common Archive Observation Model](https://www.opencadc.org/caom2/) housing structured metadata from multiple missions in a unified database, from legacy missions to currently operational missions. In this database, individual data products are organized under "observations".

* Note: for certain missions like JWST and HST, there is also a `MastMissions` API from `astroquery.mast` that can be used to query mission-specific metadata beyond that which can be made to conform to the Common Archive Observation Model. This notebook will not demonstrate the capabilities of the `MastMissions` API. Refer, instead, to [**Searching for Mission-Specific Data with Astroquery**](https://spacetelescope.github.io/mast_notebooks/notebooks/multi_mission/missions_mast_search/missions_mast_search.html).

First, let's turn on access to MAST's datasets in the AWS cloud. Downloading from AWS is at least sometimes faster, and is always preferred in order to avoid putting too much pressure on MAST's on-premise servers.

In [ ]:
Observations.enable_cloud_dataset()

# The Four-Step Data Download Process

Retrieving MAST data from AWS to your local machine can be performed with the following four-step process:

* **Step 1**: Retrieve observation metadata matching your query criteria
* **Step 2**: Retrieve metadata for the individual data products that comprise those observations
* **Step 3**: (Optional) Filter the data products based on further product-level criteria
* **Step 4**: Download the files from AWS to your local machine

### Step 1

First, let's retrieve observations in a sky coordinate range of interest. One way to search observations by coordinates is by giving `Observations.query_criteria()` a box defining the search area, consisting of two coordinates for the right ascension range and two coordinates for the declination range. You can also supply any number of missions, and various other metadata constraints. In this case, we'll retrieve both GALEX and Pan-STARRS observations. The output is an `astropy` table.

In [ ]:
obs = Observations.query_criteria(s_ra=[30.2, 31.2], s_dec=[-10.25, -9.25], obs_collection=["GALEX", "PS1"])
print(f'We retrieved {len(obs)} observations.')

If you would like to filter by other parameters, see the other filter parameters below.

In [ ]:
# NOTE: Use this line if you want to get app the parameters and the details.
# Observations.get_metadata("observations").pprint(max_lines=-1, max_width=-1)

# NOTE: Use this line just to get the parameters and their names.
Observations.get_metadata('observations')['Column Name'].pprint(max_lines=-1)

### Step 2

Now, we can retrieve the individual data products organized under those observations.

In [ ]:
prod = Observations.get_product_list(obs)
print(f'We retrieved {len(prod)} data products.')
# prod is another astropy table

### Step 3

Now we can use `Observations.filter_products()` to filter for specific data products. This function can filter on `obs_collection` (mission), `productType`,`productSubGroupDescription`, `productGroupDescription`, and `mrp_only`, among numerous other parameters described here: [product field descriptions](https://masttest.stsci.edu/api/v0/_productsfields.html). 

`mrp_only` means to request only data products identified by MAST as the main "Minimum Recommended" products in each observation. For example, in Pan-STARRS (PS1), limiting your results to MRP products excludes the individual-epoch warp images, and various other ancillary files.

For **GALEX**, the range of possible values for these parameters include:

* productType: *AUXILIARY*, *CATALOG*, *INFO*, *PREVIEW*, *SCIENCE*, *THUMBNAIL*
* productSubGroupDescription: *Catalog Only*, *Imaging Only*, *Spectra Only*, *Spectral Image Strips Only*, *Whole Field Images Only*
* productGroupDescription: *Minimum Recommended Products*
* mrp_only: *True*, *False*.

For **Pan-STARRS (PS1)**, the range of possible values for these parameters include:

* productType: *AUXILIARY*, *CATALOG*, *INFO*, *SCIENCE*
* productSubGroupDescription: N/A
* productGroupDescription: *Minimum Recommended Products*
* mrp_only: *True*, *False*

Note that *productSubGroupDescription* and *productGroupDescription* are generally not needed when filtering for Pan-STARRS products.

Let's start with GALEX:

In [ ]:
filt_prod_galex = Observations.filter_products(
    prod,
    obs_collection="GALEX",
    productType="SCIENCE",
    productSubGroupDescription="Imaging Only",
    productGroupDescription="Minimum Recommended Products",
    mrp_only=True
)

# Shows how many files are left after applying the filter.
print(f'We are left with {len(filt_prod_galex)} files.')

# Shows the filtered table of data products.
display(filt_prod_galex)

And now let's try Pan-STARRS:

In [ ]:
filt_prod_ps1 = Observations.filter_products(
    prod,
    obs_collection="PS1",
    mrp_only=True
)

# Show how many files are left after applying the filter.
print(f'We are left with {len(filt_prod_ps1)} files.')

# Show the first few rows of the filtered table of data products.
display(filt_prod_ps1[0:3])

### Step 4

Download the files you need to your local computer using `Observations.download_products`! For example, for GALEX:

In [ ]:
Observations.download_products(filt_prod_galex, cloud_only=True)

Note that, because you turned on `Observations.enable_cloud_dataset` earlier, `download_products` will attempt to download every file from the AWS S3 bucket no matter what you set for `cloud_only`. However, if you leave `cloud_only=False` as per the default, astroquery will download a file from MAST's on-premise server if it can't find the file in AWS. If you turn on `cloud_only=True` as above, astroquery will skip downloading any file that it can't find in AWS.

# About this Notebook

* **Authors**: Yingquan Li, Bernie Shao
* **Keywords**: GALEX, Pan-STARRS, Bulk Download, Python, AWS
* **Updated On**: 2025-04-23
* **References**: [Missions Mast Search](https://github.com/spacetelescope/mast_notebooks/blob/main/notebooks/multi_mission/missions_mast_search/missions_mast_search.ipynb)

For support, please contact the Archive HelpDesk at archive@stsci.edu.